In [1]:
import mojimoji
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
pd.set_option('display.max_columns',100)
# import japanize_matplotlib

In [3]:
train_path='./input/train.csv'  
test_path='./input/train.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
train_y = train['賃料']
data = pd.concat([train.drop(columns='賃料'), test], axis=0)

/home/higuchi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## Preprocessing

### 所在地の正規化

In [4]:
def preprocess_location(data,in_col='所在地',out_cols=['所在丁','地域']):
    def preprocess(x):
        x = mojimoji.zen_to_han(x, kana=False)
        x = re.sub('[()（）()/(/)/（/）/(/)]', '', x)
        #5番目8丁目3番→5-8-3みたいにする
        x = re.sub('丁目|番$|ー', '-', x)
        x = re.sub('-$|号|未定|予定|東京都', '', x)
        #ゴミの削除
        x= re.sub(re.compile("[!-/:-@[-`{-~]"), '', x)
        #空白文字の削除
        x=''.join(x.split())
        return x
    
    #数字以降全部消去
    def delete_num(x):
        x= re.sub('\d.*','',x)
        return x
    #5-6-4 ->5にする
    def delete_second_num(x):
        x = re.sub('(?<=-)(.*)', '', x)
        x = re.sub('-', '', x)
        return x

    #全部残す、丁まで残す、丁以降全部消す
    data[in_col]=data[in_col].apply(preprocess)
    data[out_cols[0]] = data[in_col].apply(delete_second_num)
    data[out_cols[1]] = data[in_col].apply(delete_num)
    return data




### 築年数→築年数と築ヶ月を生成

In [17]:
def preprocess_build_age(data,in_col='築年数',out_cols=['築年数','築ヶ月数']):
    data2=data.copy()
    #年で分割、dfとして一時的に保管
    tmp_df = data[in_col].str.split('年', expand=True)
    tmp_df.columns=['year','month']
    #ヶ月以降の文字列を除去
    tmp_df['month'] = tmp_df['month'].str.split('ヶ月', expand=True)[0]
    #新築は0年目とする
    tmp_df.loc[tmp_df['year'] == '新築', 0] = 0
    tmp_df=tmp_df.fillna(-1).astype(int)
    #欠損値を平均で埋めて整数にする
    tmp_df=tmp_df.replace({-1:np.nan}).fillna(tmp_df.mean()).astype(int)
    # tmp_df = tmp_df.fillna(tmp_df.median())
    tmp_df['month2'] = tmp_df['year'] * 12 + tmp_df['month']
    data2[out_cols[0]] = tmp_df['year']
    data2[out_cols[1]] = tmp_df['month2']
    return data2



### 所在階
所在階→地下の人は地下列に入力

In [18]:
def preprocess_floor(data,in_col='所在階',out_cols=['戸建て','総階数', '地下階数', '所在階']):
    data2=data.copy()
    
    data2[out_cols[0]] = data2[in_col].apply(lambda x: 0 if '／' in str(x) else 1)
    
    # 所在階と合計階数と地下階数に分ける
    data2[out_cols[1]] = [int(re.findall('(\d+階建)', i)[0][:-2]) if len(re.findall('(\d+階建)', i)) != 0 else np.nan  for i in data2[in_col].astype(str)]
    data2[out_cols[2]] = [int(re.findall('(地下\d+階)', i)[0][2:-1]) if len(re.findall('(地下\d+階)', i)) != 0 else 0  for i in data2[in_col].astype(str)]
    data2[out_cols[3]] = [int(re.findall('(\d+階[^建][^)])', i)[0][:-3]) if len(re.findall('(\d+階[^建][^)])', i)) != 0 else np.nan  for i in data2[in_col].astype(str)]
    return data2
    
#     tmp_df = data2[in_col].str.split('／', expand=True)
#     # 階がない奴は階→階建てに変換
#     tmp_df.loc[tmp_df[1].isnull(), 1] = tmp_df.loc[tmp_df[1].isnull(), 0]
#     tmp_df.loc[tmp_df[1].isnull(), 0] = np.nan

#     # (地下ほげ階)みたいな表現を削除
#     tmp_df[1] = tmp_df[1].str.split('（', expand=True)[0].str.split('階', expand=True)[0]
#     tmp_df[0] = tmp_df[0].str.split('（', expand=True)[0].str.split('階', expand=True)[0]

#     # 地下かどうか判定
#     tmp_df['is_under'] = 0
#     tmp_df.loc[tmp_df[0] == -1, 0] = '-1'
#     tmp_df = tmp_df.fillna('-1')

#     tmp_df.loc[tmp_df[0].str.contains('地下'), 'is_under'] = 1
#     tmp_df[0] = tmp_df[0].str.strip('地下')
#     tmp_df.columns = ['階', '階建て', '地下']
#     data2 = pd.concat([data2, tmp_df], axis=1)


### タブ区切りデータ
バストイレ、キッチン、放送通信、室内設備  
タブで区切る。要素数も追加する

In [19]:
def preprocess_tab_data(data,in_col='バス・トイレ',out_cols=['バス・トイレ','バス・トイレ数']):
    data2=data.copy()

    zen2han=lambda x :mojimoji.zen_to_han(x, kana=False)
    #リスト内の空白要素を削除
    list_strip =lambda list_:[x for x in list_ if x]
    #欠損値埋め→半角変換→スラッシュ削除
    data2[in_col] = data2[in_col].fillna('None').apply(zen2han).str.replace('/', '')
    data2[out_cols[0]] = data2[in_col].str.split('\t').apply(list_strip)
    data2[out_cols[1]] = data2[in_col].apply(lambda x: len(x))
    return data2


### 面積

In [20]:
def preprocess_area(data,in_col='面積',out_cols=['面積','畳']):
    data2=data.copy()
    data2[out_cols[0]] = data2[in_col].str.split('m2', expand=True)[0]
    data2[out_cols[0]] = data2[out_cols[0]].astype(float)
    data2[out_cols[1]] = data2[in_col].apply(lambda x: int(x/1.45))
    return data2

### 間取り
普通にラベルエンコーディングでもいいかも

In [21]:
def preprocess_floor_type(data,in_col='間取り',out_cols=['L', 'R', 'D', 'K', 'S','LDK','DK','部屋数']):
    data2=data.copy()
    for alphabet in out_cols[:-1]:
        data2[alphabet]=0
        data2.loc[data2[in_col].str.contains(alphabet), alphabet] = 1
    #部屋数
    data2[out_cols[-1]] = data2[in_col].apply(lambda x: re.sub("\\D", "", x))
    data2[out_cols[-1]]=data2[out_cols].sum(axis=1)
    return data2

### アクセス

In [22]:
def preprocess_access(data,in_col='アクセス',out_cols=['線名','駅名']):
    data2=data.copy()
    def separate_all(x):
        list_ = re.split('\t|\u3000|・|：', x)
        #カッコとカッコ内の言葉を除去
        list_ = [re.sub('\(.*?\)|\（.*?\）', '', x)for x in list_]
        list_ = [x for x in list_ if x]
        return list_

    def extract_rails(List, word='線'):
        return [x.split(word)[0] + word for x in List if re.search(word, x)]

    def extract_stations(List, word='駅'):
        List = [x.split(word)[0] + word for x in List if re.search(word, x)]
        New_List = []
        for x in List:
            if re.search('線', x):
                x = x.split('線')[1]
            if re.search('(＼|バス)', x):
                x = re.split('(＼|バス)', x)[1]
            New_List.append(x)
        return New_List

    data2[in_col] = data2[in_col].apply(separate_all)
    data2[out_cols[0]] = data2[in_col].apply(extract_rails, word='線')
    data2[out_cols[1]] = data2[in_col].apply(extract_stations, word='駅')
    return data2

### 周辺環境

In [23]:
def preprocess_spot(data,in_col='周辺環境',out_cols=['小学校', '大学', '公園', '飲食店', 'スーパー', 'ドラッグストア', '郵便局', '病院', '幼稚園・保育園', '銀行','コンビニ']):
    data2=data.copy()
    data2[in_col]=data2[in_col].fillna('None')
    for spot in out_cols:
        data2[spot] = data2[in_col].str.contains(spot) * 1
        data2[str(spot)+'_dis'] = data2['周辺環境'].apply(lambda x: int(str(x).split()[str(x).split().index(spot)+1].replace('m','')) if spot in str(x).split() else 0)     
    
    return data2

### 駐車場

In [24]:
def preprocess_parking(data,in_col='駐車場',out_cols=['駐輪場\t空有', '駐車場\t空有', 'バイク置き場\t空有', '駐車場\t近隣']):
    data2=data.copy()
    data2[in_col]=data2[in_col].fillna('None')
    for spot in out_cols:
        data2[''.join(spot.split('\t'))] = data2[in_col].str.contains(spot) * 1
    return data2

### 契約期間

In [25]:
def preprocess_contract_year(data,in_col='契約期間',out_cols=['契約期間']):
    data2=data.copy()
    data2[in_col] = data2[in_col].str.split('\t', expand=True)[0].str.split('年', expand=True)[0].fillna('0')
    def pp_contract_year(x):
        try:
            x = int(x)
            if x > 2000:
                return x - 2019
            return x
        except:
            return 0

    data2[in_col] = data2[in_col].apply(pp_contract_year)
    return data2

In [26]:

data2=data.copy()
data2=preprocess_build_age(data2)
data2=regax_location(data2)
data2=preprocess_floor(data2)
data2=preprocess_tab_data(data2,'バス・トイレ',['バス・トイレ','バス・トイレ数'])
data2=preprocess_tab_data(data2,'キッチン',['キッチン','キッチン数'])
data2=preprocess_tab_data(data2,'放送・通信',['放送・通信','放送・通信数'])
data2=preprocess_tab_data(data2,'室内設備',['室内設備','室内設備数'])
data2=preprocess_floor_type(data2)
data2=preprocess_area(data2)
data2=preprocess_access(data2)
data2=preprocess_spot(data2)
data2=preprocess_parking(data2)
data2=preprocess_contract_year(data2)



ValueError: invalid literal for int() with base 10: '新築'

In [15]:
data2.drop(columns=['アクセス','キッチン','バス・トイレ',

SyntaxError: unexpected EOF while parsing (<ipython-input-15-a0f1a512ee8a>, line 1)

In [111]:
data2.head(1)

,id,アクセス,キッチン,バス・トイレ,周辺環境,契約期間,室内設備,建物構造,所在地,所在階,放送・通信,方角,築年数,賃料,間取り,面積,駐車場,築年,築ヶ月数,所在丁,地域,階,階建て,地下,バス・トイレ数,キッチン数,放送・通信数,室内設備数,L,R,D,K,S,LDK,DK,部屋数,線名,駅名,小学校,大学,公園,飲食店,スーパー,ドラッグストア,郵便局,病院,幼稚園・保育園,銀行,コンビニ,駐輪場空有,駐車場空有,バイク置き場空有,駐車場近隣
0,1,"[都営三田線, 西巣鴨駅, 徒歩4分, 埼京線, 板橋駅, 徒歩14分, 都電荒川線, 西ケ...","[ガスコンロ, コンロ2口, システムキッチン, 給湯]","[専用バス, 専用トイレ, バス・トイレ別, シャワー, 浴室乾燥機, 温水洗浄便座]",【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,2,"[エアコン付, シューズボックス, バルコニー, フローリング, 室内洗濯機置場, 敷地内ご...",RC（鉄筋コンクリート）,北区滝野川3,1階／12階建,"[インターネット対応, CATV, CSアンテナ, BSアンテナ]",南東,9年9ヶ月,NaN,1K,20.01,駐輪場\t空有,9,117,北区滝野川3,北区滝野川,1,12,0,6,4,4,10,0,0,0,1,0,0,0,1,"[都営三田線, 埼京線, 都電荒川線]","[西巣鴨駅, 板橋駅, 西ケ原四丁目駅]",1,1,1,1,1,1,1,0,0,0,1,1,0,0,0
